In [2]:
#!rm -r /home/jovyan/.local/share/jupyter/kernels/nilmtk
!mamba install tensorflow-gpu==2.10 -y -q
#!mamba install tensorflow-gpu==2.11.0 -y -q
!pip uninstall -y -q nilmtk nilm_metadata
#!pip install -q pandas networkx tables scikit-learn hmmlearn pyyaml matplotlib==3.1.3 xgboost pyts
# Trick to install NILM regardless of its dependencies
#!pip install --no-cache -U git+https://github.com/nilmtk/nilm_metadata.git
#!pip install --no-cache -U git+https://github.com/nilmtk/nilmtk.git@0.4.3
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilmtk@master
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilm_metadata@master

  Cloning https://github.com/nilmtk/nilmtk (to revision master) to /tmp/pip-req-build-p5mo8pze
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-p5mo8pze
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done
  Created wheel for nilmtk: filename=nilmtk-0.4.0.dev1+git.b2c5144-py3-none-any.whl size=279177 sha256=29a3eb7362c66a95d828cc4452a2e9d74555c237fc93be0d8f424260cffd71bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_r_p2jv/wheels/05/a4/8e/b767c3d1714f61fd30ec991c25780be8cc04c474da8205aeee
Successfully built nilmtk
  Cloning https://github.com/nilmtk/nilm_metadata (to revision master) to /tmp/pip-req-build-2ezg_axv
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilm_metadata /tmp/pip-req-build-2ezg_axv
  Resolved https://github.com/nilmtk/nilm_metadata to commit 7ed4bab9062d04cb35c6b6000b451715dc5ab4af
  Prep

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint

#from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost -q
import xgboost as xgb

import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
#import graphviz
import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from ann_visualizer.visualize import ann_viz
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
# GPU goes brrrrrrrrrrrrrrrrrrrr
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

!pip3 install tqdm -q
from tqdm import tqdm
import NUK

2023-09-20 06:30:15.414795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb, compute capability 8.0


# Preprocessing

# Models

In [6]:
# Parameters

epochs =25
window_size = 2700
batch_size = 256
NmDevices = 82
# table_of_options = [1,2,3,4]
k = 0.25
# Define a learning rate scheduler function
def scheduler(epoch, lr):
    return lr
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

RANDOM_SEED = 42
test_size = 0.2


In [40]:
data= pd.read_pickle("../../Energy_graph/data/processed_all_X_Y_watts.pkl")
data_syn = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/synthetic_test_50k_2700.pkl")
labels = pd.read_pickle("../../Energy_graph/labeles.pkl")


# Separate the tuples into X and y
X_syn = np.array([i[0] for i in data_syn])
y_syn = np.array([i[1] for i in data_syn])

X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])

# y = y.astype(int)

X = np.concatenate((X, X_syn), axis=0)
y = np.concatenate((y, y_syn), axis=0)

# X_test = np.array([i[0] for i in data_test])
# y_test = np.array([i[1] for i in data_test])



In [37]:
len(labels)

82

### PC0

In [41]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = NUK.PC0(NmDevices, window_size, 'GRU', 128, k)
    model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    lr_scheduler = LearningRateScheduler(scheduler)
    callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,patience=2, min_lr=0.0000002)
    # for class weights
    # model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights_UKD, callbacks=[lr_scheduler])
    # class_weights = NUK.class_weights_tool(y_train)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test))

    # n_labels = y_test.shape[1]
    # co_occurrence_matrix = np.zeros((n_labels, n_labels))

    # for true, pred in zip(y_test, y_pred_tf):
    #     fn_labels = np.where((true == 1) & (pred == 0))[0]  # False negatives
    #     fp_labels = np.where((true == 0) & (pred == 1))[0]  # False positives

    #     for fn in fn_labels:
    #         for fp in fp_labels:
    #             co_occurrence_matrix[fn, fp] += 1



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)


    


0it [00:00, ?it/s]

Train indices: [    0     1     2 ... 97051 97053 97054] Test indices: [    8    39    53 ... 97049 97052 97055]
Epoch 1/25
304/304 [==============================] - 9s 26ms/step - loss: nan - F1Score: 0.0209 - WeightedF1: 0.0510 - lr: 3.0000e-04
Epoch 2/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.0605 - WeightedF1: 0.1250 - lr: 3.0000e-04
Epoch 3/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.0931 - WeightedF1: 0.1758 - lr: 3.0000e-04
Epoch 4/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.1166 - WeightedF1: 0.2043 - lr: 3.0000e-04
Epoch 5/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1424 - WeightedF1: 0.2347 - lr: 3.0000e-04
Epoch 6/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1747 - WeightedF1: 0.2717 - lr: 3.0000e-04
Epoch 7/25
304/304 [==============================] - 8s 27ms/step - loss: na

1it [03:30, 210.17s/it]

Train indices: [    0     1     2 ... 97053 97054 97055] Test indices: [    4     6     7 ... 97042 97047 97050]
Epoch 1/25
304/304 [==============================] - 10s 27ms/step - loss: nan - F1Score: 0.0201 - WeightedF1: 0.0482 - lr: 3.0000e-04
Epoch 2/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.0597 - WeightedF1: 0.1219 - lr: 3.0000e-04
Epoch 3/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.0861 - WeightedF1: 0.1642 - lr: 3.0000e-04
Epoch 4/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1096 - WeightedF1: 0.1954 - lr: 3.0000e-04
Epoch 5/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1338 - WeightedF1: 0.2246 - lr: 3.0000e-04
Epoch 6/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1599 - WeightedF1: 0.2536 - lr: 3.0000e-04
Epoch 7/25
304/304 [==============================] - 8s 27ms/step - loss: n

2it [07:02, 211.54s/it]

Train indices: [    1     2     4 ... 97053 97054 97055] Test indices: [    0     3     9 ... 97039 97044 97048]
Epoch 1/25
304/304 [==============================] - 9s 27ms/step - loss: nan - F1Score: 0.0200 - WeightedF1: 0.0473 - lr: 3.0000e-04
Epoch 2/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.0593 - WeightedF1: 0.1188 - lr: 3.0000e-04
Epoch 3/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.0874 - WeightedF1: 0.1640 - lr: 3.0000e-04
Epoch 4/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1103 - WeightedF1: 0.1944 - lr: 3.0000e-04
Epoch 5/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1366 - WeightedF1: 0.2244 - lr: 3.0000e-04
Epoch 6/25
304/304 [==============================] - 8s 27ms/step - loss: nan - F1Score: 0.1643 - WeightedF1: 0.2550 - lr: 3.0000e-04
Epoch 7/25
304/304 [==============================] - 8s 27ms/step - loss: na

3it [10:32, 210.74s/it]

Train indices: [    0     2     3 ... 97052 97053 97055] Test indices: [    1    10    13 ... 97032 97045 97054]
Epoch 1/25
304/304 [==============================] - 11s 26ms/step - loss: nan - F1Score: 0.0176 - WeightedF1: 0.0431 - lr: 3.0000e-04
Epoch 2/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.0529 - WeightedF1: 0.1104 - lr: 3.0000e-04
Epoch 3/25
304/304 [==============================] - 8s 26ms/step - loss: nan - F1Score: 0.0829 - WeightedF1: 0.1607 - lr: 3.0000e-04
Epoch 4/25
 17/304 [>.............................] - ETA: 7s - loss: nan - F1Score: 0.0854 - WeightedF1: 0.1591  

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns

# save with pickle
with open('predictions_50k_mixed.pkl', 'wb') as f:
    pickle.dump(predictions, f)

# plt.figure(figsize=(20, 20))
# sns.heatmap(co_occurrence_matrix, annot=True, fmt='g', cmap='Blues')
# plt.xlabel('False Positive Labels')
# plt.ylabel('False Negative Labels')
# plt.title('Co-occurrence of False Positives and False Negatives')
# plt.show()

### Random forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
jobs = 110
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []

for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize the Random Forest classifier wrapped in a MultiOutputClassifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=jobs)
    # classifier = MultiOutputClassifier(forest, n_jobs=-1)
    classifier.fit(X_train, y_train)

    # Predict the labels
    y_pred = classifier.predict(X_test)

    # Evaluate the model using classification report
    report = classification_report(y_test, y_pred, target_names=labels, zero_division=0, output_dict=True)

    evaluation_results.append(report)


for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_staticLR_LSTM_watts_mixed100k.csv")
df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    


0it [00:00, ?it/s]

Train indices: [     1      2      3 ... 147053 147054 147055] Test indices: [     0      4     12 ... 147049 147050 147052]


In [ ]:
evaluation_results = []
model = NUK.PC0(NmDevices, window_size, 'GRU',128, k)
model.build((len(y), window_size, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
lr_scheduler = LearningRateScheduler(scheduler)

model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])


y_pred = model.predict(X_test)
y_pred_tf = (y_pred > 0.5)
report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)

# report_DF = NUK.ClassificationReportToDF(report, labels)  
# report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
df = pd.DataFrame(report)

In [12]:

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_staticLR_GRU_watts_100k_unmetered.csv")
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    
# evaluation_results+ evaluation_results[1].to_